# Exploratory Data Analysis
    
The goal of this competition is to use seismic signals to predict the timing of laboratory earthquakes. The data comes from a well-known experimental set-up used to study earthquake physics.

- The **training data** is a single, continuous segment of experimental data. 
- The **test data** consists of a folder containing many small segments. The data within each test file is continuous, but the test files do not represent a continuous segment of the experiment; thus, the predictions cannot be assumed to follow the same regular pattern seen in the training file.

For each *seg_id* in the test folder, you should predict a single *time_to_failure* corresponding to the time between the last row of the segment and the next laboratory earthquake.

[Scores](https://docs.google.com/spreadsheets/d/1AWLPZfryFWx6wM9fxmMmGm31R4B9PMrZzLp71ZtlcRk/edit#gid=0)

In [1]:
# Convenience
import gc
import warnings
from tqdm import tqdm
from tqdm import tqdm_notebook

import random
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import NuSVR
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression

# Machine Learning libraries
import xgboost as xgb
import lightgbm as lgb
from sklearn import linear_model
from sklearn import model_selection
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error, roc_curve, auc

# Signal processing
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

# Keras
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

warnings.filterwarnings("ignore")

random.seed(1)

pd.options.display.precision = 15
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
pd.options.display.width = 1000

Using TensorFlow backend.


In [2]:
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Starting memory usage: {:5.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Reduced memory usage: {:5.2f} MB ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

[Earthquakes FE. More features and samples](https://www.kaggle.com/artgor/earthquakes-fe-more-features-and-samples)<br/>
[Seismic data EDA and baseline](https://www.kaggle.com/artgor/seismic-data-eda-and-baseline)<br/>
[Shaking Earth](https://www.kaggle.com/allunia/shaking-earth)

## Loading data

In [9]:
df_train = pd.read_csv('input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32}) # nrows=300_000
df_train = reduce_memory_usage(df_train)
print("{:,} records and {} features in train set.".format(df_train.shape[0], df_train.shape[1]))

Starting memory usage:  1.72 MB
Reduced memory usage:  1.14 MB (33.3% reduction)
300,000 records and 2 features in train set.


## Feature engineering
Inspired by [Andrews Script plus a Genetic Program Model](https://www.kaggle.com/scirpus/andrews-script-plus-a-genetic-program-model)<br/>
[Understanding and parameter setting of STA/LTA trigger
algorithm](http://gfzpublic.gfz-potsdam.de/pubman/item/escidoc:4097:3/component/escidoc:4098/IS_8.1_rev1.pdf)

In [21]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

def classic_sta_lta(x, length_sta, length_lta):
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
    sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta

In [22]:
def extract_features(df_input, segment_size = 150_000):
    segments = int(np.floor(df_input.shape[0] / segment_size))
    print('Number of segments: {:,}'.format(segments))

    X_out = pd.DataFrame(index=range(segments), dtype=np.float64)
    y_out = pd.DataFrame(index=range(segments), dtype=np.float64)

    for segment in tqdm_notebook(range(segments)):
        seg = df_input.iloc[segment*segment_size:segment*segment_size+segment_size]
        x = pd.Series(seg['acoustic_data'].values)
        if seg['time_to_failure']:
            y_out.loc[segment, 'time_to_failure'] = seg['time_to_failure'].values[-1]

        X_out.loc[segment, 'mean'] = x.mean()
        X_out.loc[segment, 'std'] = x.std()
        X_out.loc[segment, 'max'] = x.max()
        X_out.loc[segment, 'min'] = x.min()

        X_out.loc[segment, 'mean_change_abs'] = np.mean(np.diff(x))
        X_out.loc[segment, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
        X_out.loc[segment, 'abs_max'] = np.abs(x).max()
        X_out.loc[segment, 'abs_min'] = np.abs(x).min()

        X_out.loc[segment, 'std_first_50000'] = x[:50000].std()
        X_out.loc[segment, 'std_last_50000'] = x[-50000:].std()
        X_out.loc[segment, 'std_first_10000'] = x[:10000].std()
        X_out.loc[segment, 'std_last_10000'] = x[-10000:].std()

        X_out.loc[segment, 'avg_first_50000'] = x[:50000].mean()
        X_out.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
        X_out.loc[segment, 'avg_first_10000'] = x[:10000].mean()
        X_out.loc[segment, 'avg_last_10000'] = x[-10000:].mean()

        X_out.loc[segment, 'min_first_50000'] = x[:50000].min()
        X_out.loc[segment, 'min_last_50000'] = x[-50000:].min()
        X_out.loc[segment, 'min_first_10000'] = x[:10000].min()
        X_out.loc[segment, 'min_last_10000'] = x[-10000:].min()

        X_out.loc[segment, 'max_first_50000'] = x[:50000].max()
        X_out.loc[segment, 'max_last_50000'] = x[-50000:].max()
        X_out.loc[segment, 'max_first_10000'] = x[:10000].max()
        X_out.loc[segment, 'max_last_10000'] = x[-10000:].max()

        X_out.loc[segment, 'max_to_min'] = x.max() / np.abs(x.min())
        X_out.loc[segment, 'max_to_min_diff'] = x.max() - np.abs(x.min())
        X_out.loc[segment, 'count_big'] = len(x[np.abs(x) > 500])
        X_out.loc[segment, 'sum'] = x.sum()

        X_out.loc[segment, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(x[:50000]) / x[:50000][:-1]))[0])
        X_out.loc[segment, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(x[-50000:]) / x[-50000:][:-1]))[0])
        X_out.loc[segment, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(x[:10000]) / x[:10000][:-1]))[0])
        X_out.loc[segment, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(x[-10000:]) / x[-10000:][:-1]))[0])

        X_out.loc[segment, 'q95'] = np.quantile(x, 0.95)
        X_out.loc[segment, 'q99'] = np.quantile(x, 0.99)
        X_out.loc[segment, 'q05'] = np.quantile(x, 0.05)
        X_out.loc[segment, 'q01'] = np.quantile(x, 0.01)

        X_out.loc[segment, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
        X_out.loc[segment, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
        X_out.loc[segment, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
        X_out.loc[segment, 'abs_q01'] = np.quantile(np.abs(x), 0.01)

        X_out.loc[segment, 'trend'] = add_trend_feature(x)
        X_out.loc[segment, 'abs_trend'] = add_trend_feature(x, abs_values=True)
        X_out.loc[segment, 'abs_mean'] = np.abs(x).mean()
        X_out.loc[segment, 'abs_std'] = np.abs(x).std()

        X_out.loc[segment, 'mad'] = x.mad()
        X_out.loc[segment, 'kurt'] = x.kurtosis()
        X_out.loc[segment, 'skew'] = x.skew()
        X_out.loc[segment, 'med'] = x.median()

        X_out.loc[segment, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
        X_out.loc[segment, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
        X_out.loc[segment, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
        X_out.loc[segment, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
        X_out.loc[segment, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
        X_out.loc[segment, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
        X_out.loc[segment, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
        X_out.loc[segment, 'Moving_average_1500_mean'] = x.rolling(window=1500).mean().mean(skipna=True)
        X_out.loc[segment, 'Moving_average_3000_mean'] = x.rolling(window=3000).mean().mean(skipna=True)
        X_out.loc[segment, 'Moving_average_6000_mean'] = x.rolling(window=6000).mean().mean(skipna=True)
        ewma = pd.Series.ewm
        X_out.loc[segment, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
        X_out.loc[segment, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
        X_out.loc[segment, 'exp_Moving_average_30000_mean'] = ewma(x, span=6000).mean().mean(skipna=True)
        no_of_std = 2
        X_out.loc[segment, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
        X_out.loc[segment,'MA_700MA_BB_high_mean'] = (X_out.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_out.loc[segment, 'MA_700MA_std_mean']).mean()
        X_out.loc[segment,'MA_700MA_BB_low_mean'] = (X_out.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_out.loc[segment, 'MA_700MA_std_mean']).mean()
        X_out.loc[segment, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
        X_out.loc[segment,'MA_400MA_BB_high_mean'] = (X_out.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_out.loc[segment, 'MA_400MA_std_mean']).mean()
        X_out.loc[segment,'MA_400MA_BB_low_mean'] = (X_out.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_out.loc[segment, 'MA_400MA_std_mean']).mean()
        X_out.loc[segment, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()

        X_out.loc[segment, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
        X_out.loc[segment, 'q999'] = np.quantile(x, 0.999)
        X_out.loc[segment, 'q001'] = np.quantile(x, 0.001)
        X_out.loc[segment, 'ave10'] = stats.trim_mean(x, 0.1)

        for windows in [10, 100, 1000]:
            x_roll_std = x.rolling(windows).std().dropna().values
            x_roll_mean = x.rolling(windows).mean().dropna().values

            X_out.loc[segment, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
            X_out.loc[segment, 'std_roll_std_' + str(windows)] = x_roll_std.std()
            X_out.loc[segment, 'max_roll_std_' + str(windows)] = x_roll_std.max()
            X_out.loc[segment, 'min_roll_std_' + str(windows)] = x_roll_std.min()
            X_out.loc[segment, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
            X_out.loc[segment, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
            X_out.loc[segment, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
            X_out.loc[segment, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
            X_out.loc[segment, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
            X_out.loc[segment, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
            X_out.loc[segment, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()

            X_out.loc[segment, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
            X_out.loc[segment, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
            X_out.loc[segment, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
            X_out.loc[segment, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
            X_out.loc[segment, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
            X_out.loc[segment, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
            X_out.loc[segment, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
            X_out.loc[segment, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
            X_out.loc[segment, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
            X_out.loc[segment, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
            X_out.loc[segment, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
    
    return X_out, y_out

In [23]:
X_tr, y_tr = extract_features(df_train)
print('{:,} samples in new train data and {:,} columns.'.format(X_tr.shape[0], X_tr.shape[1]))

Number of segments: 2


In [ ]:
submission = pd.read_csv('input/sample_submission.csv', index_col='seg_id')
X_te, _ = extract_features(submission)

'''
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)

for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('input/test/' + seg_id + '.csv')
    
    x = pd.Series(seg['acoustic_data'].values)
    X_test.loc[seg_id, 'mean'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
        
    X_test.loc[seg_id, 'mean_change_abs'] = np.mean(np.diff(x))
    X_test.loc[seg_id, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_test.loc[seg_id, 'abs_max'] = np.abs(x).max()
    X_test.loc[seg_id, 'abs_min'] = np.abs(x).min()
    
    X_test.loc[seg_id, 'std_first_50000'] = x[:50000].std()
    X_test.loc[seg_id, 'std_last_50000'] = x[-50000:].std()
    X_test.loc[seg_id, 'std_first_10000'] = x[:10000].std()
    X_test.loc[seg_id, 'std_last_10000'] = x[-10000:].std()
    
    X_test.loc[seg_id, 'avg_first_50000'] = x[:50000].mean()
    X_test.loc[seg_id, 'avg_last_50000'] = x[-50000:].mean()
    X_test.loc[seg_id, 'avg_first_10000'] = x[:10000].mean()
    X_test.loc[seg_id, 'avg_last_10000'] = x[-10000:].mean()
    
    X_test.loc[seg_id, 'min_first_50000'] = x[:50000].min()
    X_test.loc[seg_id, 'min_last_50000'] = x[-50000:].min()
    X_test.loc[seg_id, 'min_first_10000'] = x[:10000].min()
    X_test.loc[seg_id, 'min_last_10000'] = x[-10000:].min()
    
    X_test.loc[seg_id, 'max_first_50000'] = x[:50000].max()
    X_test.loc[seg_id, 'max_last_50000'] = x[-50000:].max()
    X_test.loc[seg_id, 'max_first_10000'] = x[:10000].max()
    X_test.loc[seg_id, 'max_last_10000'] = x[-10000:].max()
    
    X_test.loc[seg_id, 'max_to_min'] = x.max() / np.abs(x.min())
    X_test.loc[seg_id, 'max_to_min_diff'] = x.max() - np.abs(x.min())
    X_test.loc[seg_id, 'count_big'] = len(x[np.abs(x) > 500])
    X_test.loc[seg_id, 'sum'] = x.sum()
    
    X_test.loc[seg_id, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(x[:50000]) / x[:50000][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(x[-50000:]) / x[-50000:][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(x[:10000]) / x[:10000][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(x[-10000:]) / x[-10000:][:-1]))[0])
    
    X_test.loc[seg_id, 'q95'] = np.quantile(x,0.95)
    X_test.loc[seg_id, 'q99'] = np.quantile(x,0.99)
    X_test.loc[seg_id, 'q05'] = np.quantile(x,0.05)
    X_test.loc[seg_id, 'q01'] = np.quantile(x,0.01)
    
    X_test.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
    X_test.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
    X_test.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
    X_test.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(x), 0.01)
    
    X_test.loc[seg_id, 'trend'] = add_trend_feature(x)
    X_test.loc[seg_id, 'abs_trend'] = add_trend_feature(x, abs_values=True)
    X_test.loc[seg_id, 'abs_mean'] = np.abs(x).mean()
    X_test.loc[seg_id, 'abs_std'] = np.abs(x).std()
    
    X_test.loc[seg_id, 'mad'] = x.mad()
    X_test.loc[seg_id, 'kurt'] = x.kurtosis()
    X_test.loc[seg_id, 'skew'] = x.skew()
    X_test.loc[seg_id, 'med'] = x.median()
    
    X_test.loc[seg_id, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
    X_test.loc[seg_id, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
    X_test.loc[seg_id, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
    X_test.loc[seg_id, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
    X_test.loc[seg_id, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
    X_test.loc[seg_id, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
    X_test.loc[seg_id, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_1500_mean'] = x.rolling(window=1500).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_3000_mean'] = x.rolling(window=3000).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_6000_mean'] = x.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_test.loc[seg_id, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
    X_test.loc[seg_id, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
    X_test.loc[seg_id, 'exp_Moving_average_30000_mean'] = ewma(x, span=6000).mean().mean(skipna=True)
    no_of_std = 2
    X_test.loc[seg_id, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
    X_test.loc[seg_id,'MA_700MA_BB_high_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X_test.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X_test.loc[seg_id,'MA_700MA_BB_low_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X_test.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X_test.loc[seg_id, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
    X_test.loc[seg_id,'MA_400MA_BB_high_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X_test.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X_test.loc[seg_id,'MA_400MA_BB_low_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X_test.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X_test.loc[seg_id, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
    
    X_test.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_test.loc[seg_id, 'q999'] = np.quantile(x,0.999)
    X_test.loc[seg_id, 'q001'] = np.quantile(x,0.001)
    X_test.loc[seg_id, 'ave10'] = stats.trim_mean(x, 0.1)
    
    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
        
        X_test.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X_test.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X_test.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X_test.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X_test.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_test.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_test.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_test.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X_test.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X_test.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X_test.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X_test.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X_test.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X_test.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_test.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_test.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X_test.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_test.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X_test.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X_test.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X_test.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X_test.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
'''

### Saving features

In [ ]:
X_tr_all = pd.concat([X_tr, y_tr], axis=1)

In [ ]:
X_tr_all = reduce_memory_usage(X_tr_all)

del df_train
gc.collect()

In [ ]:
X_tr_all.to_csv('preprocessed/segment_features_train_150k.csv')

In [ ]:
np.abs(X_tr.corrwith(y_tr['time_to_failure'])).sort_values(ascending=False).head(10)

In [ ]:
plt.figure(figsize=(44, 24))
cols = list(np.abs(X_tr.corrwith(y_tr['time_to_failure'])).sort_values(ascending=False).head(24).index)
for i, col in enumerate(cols):
    plt.subplot(6, 4, i + 1)
    plt.plot(X_tr[col], color='blue')
    plt.title(col)
    ax1.set_ylabel(col, color='b')

    ax2 = ax1.twinx()
    plt.plot(y_tr, color='g')
    ax2.set_ylabel('time_to_failure', color='g')
    plt.legend([col, 'time_to_failure'], loc=(0.875, 0.9))
    plt.grid(False)

In [ ]:
X_test.to_csv('preprocessed/segment_features_test_150k.csv')

### Load features

In [3]:
X_tr = pd.read_csv('preprocessed/segment_features_train_150k.csv', index_col=0)
X_tr = reduce_memory_usage(X_tr)
print("{:,} records and {} features in train set.".format(X_tr.shape[0], X_tr.shape[1]))

Starting memory usage:  4.48 MB
Reduced memory usage:  1.21 MB (73.0% reduction)
4,194 records and 139 features in train set.


In [4]:
X_test = pd.read_csv('preprocessed/segment_features_test_150k.csv', index_col=0)
X_test = reduce_memory_usage(X_test)
print("{:,} records and {} features in train set.".format(X_test.shape[0], X_test.shape[1]))

Starting memory usage:  2.78 MB
Reduced memory usage:  0.75 MB (73.0% reduction)
2,624 records and 138 features in train set.


In [5]:
y_train = X_tr['time_to_failure']
X_train = X_tr.drop('time_to_failure', axis=1)

## Feature selection
- **Reduces Overfitting**: Less redundant data means less opportunity to make decisions based on noise.
- **Improves Accuracy**: Less misleading data means modeling accuracy improves.
- **Reduces Training Time**: Less data means that algorithms train faster.
### SelectKBest

In [ ]:
test = SelectKBest(score_func=mutual_info_regression, k=5)
fit = test.fit(X_train, y_train)

features = fit.transform(X_train)
mask = fit.get_support()

In [ ]:
pd.DataFrame(data={
    'feature_name': list(X_train.columns[mask]),
    'score': fit.scores_[mask]
}).sort_values(by=['score'], ascending=False)

## Training

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

### Machine Learning Algorithms

In [ ]:
MLA = [
    # Linear models
    linear_model.LinearRegression(),
    linear_model.Ridge(),
    linear_model.Lasso(alpha=0.1),
    linear_model.ElasticNet(),
    linear_model.LassoLars(alpha=.1),
    linear_model.BayesianRidge(),
    linear_model.SGDRegressor(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.RANSACRegressor(),
    #linear_model.TheilSenRegressor(),
    linear_model.HuberRegressor(),

    # SVR
    NuSVR(),

    # LightGBM
    lgb.LGBMRegressor(learning_rate=0.05, n_estimators = 50000, eval_metric='mae', early_stopping_rounds=200),
    
    # CatBoost
    CatBoostRegressor(iterations=3000, learning_rate=0.03, eval_metric='MAE', verbose=False)
]

cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .25, train_size = .75, random_state = 0 )

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = X_train

#index through MLA and save performance to table
row_index = 0
for alg in MLA:
    #set name and parameters
    MLA_name = alg.__class__.__name__
    print(MLA_name)
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, X_train_scaled, y_train, cv  = cv_split)

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!

    #save MLA predictions - see section 6 for usage
    alg.fit(X_train_scaled, y_train)
    MLA_predict[MLA_name] = alg.predict(X_train_scaled)

    row_index+=1
    
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = True, inplace = True)

In [ ]:
MLA_compare

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(x='MLA Test Accuracy Mean', y = 'MLA Name', data = MLA_compare, palette=sns.cubehelix_palette(len(MLA_compare), start=2, rot=0, dark=0.2, light=.85, reverse=False))

plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')
plt.ylabel('Algorithm')
plt.tight_layout()

### RNN
[RNN starter for huge time series](https://www.kaggle.com/mayer79/rnn-starter-for-huge-time-series)

In [ ]:
def extract_features(z):
     return np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1)]

In [ ]:
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)

    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3

    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations. 
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]

In [ ]:
n_features = create_X(df_train[0:150000].values).shape[1]
print("Our RNN is based on {} features".format(n_features))

In [ ]:
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
    if max_index is None:
        max_index = len(data) - 1

    while True:
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)

        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )

        for j, row in enumerate(rows):
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            targets[j] = data[row - 1, 1]
        yield samples, targets

In [ ]:
batch_size = 32

# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877
df_train.values[second_earthquake, 1]

# Initialize generators
train_gen = generator(df_train.values, batch_size=batch_size) # Use this for better score
# train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
valid_gen = generator(df_train.values, batch_size=batch_size, max_index=second_earthquake)

In [ ]:
cb = [ModelCheckpoint("output/rnn.hdf5", save_best_only=True, period=3)]

model = Sequential()
model.add(CuDNNGRU(48, input_shape=(None, n_features)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.summary()

# Compile and fit model
model.compile(optimizer=adam(lr=0.0005), loss="mae")

history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=0,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)

In [ ]:
import matplotlib.pyplot as plt

def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None

perf_plot(history)

In [ ]:
submission = pd.read_csv('input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

# Load each test data, create the feature matrix, get numeric prediction
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

# Save
submission.to_csv('submission.csv')

- https://www.kaggle.com/jsaguiar/seismic-data-exploration
- https://www.kaggle.com/mjbahmani/probability-of-earthquake
- Feature selection
- RNN
- Ensembling (stacking, blending)

https://www.kaggle.com/scirpus/andrews-script-plus-a-genetic-program-model
https://www.kaggle.com/gpreda/lanl-earthquake-eda-and-prediction
https://www.kaggle.com/nisargpatel/time-series-garch-ts-regression
https://www.kaggle.com/zikazika/useful-new-features-and-a-optimised-model
https://www.kaggle.com/mayer79/rnn-starter-for-huge-time-series
https://www.kaggle.com/jsaguiar/seismic-data-exploration